## Spinner 1 - asyncio with decorators and yield from

In [28]:
import asyncio
import itertools
import sys

In [29]:
@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(0.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))

In [30]:
@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(3)
    return 42

In [31]:
@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

In [32]:
def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)

## Spinner2 with async, await

In [22]:
async def spin2(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            await asyncio.sleep(0.05)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))

In [23]:
async def slow_function2():
    await asyncio.sleep(3)
    return 42

In [24]:
async def supervisor2():
    spinner = asyncio.ensure_future(spin2('thinking!'))
    print('spinner object:', spinner)
    result = await slow_function2()
    spinner.cancel()
    return result

In [25]:
def main2():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor2())
    loop.close()
    print('Answer:', result)

## aiohttp

In [2]:
import asyncio
import aiohttp
from flags import BASE_URL, save_flag, show, main

In [6]:
@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    image = yield from resp.read()
    return image

In [5]:
@asyncio.coroutine
def download_one(cc):
    image = yield from get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

In [ ]:
def download_many(cc_list):
    loop = asyncio.get_event_loop()
    tasklist = [download_one(cc) for cc in sorted(cc_list)]
    wait_coro = asyncio.wait(tasklist)
    res, _ = loop.run_until_complete(wait_coro)
    loop.close()
    return len(res)

In [7]:
from enum import Enum

In [10]:
help(Enum)

Help on class Enum in module enum:

class Enum(builtins.object)
 |  Generic enumeration.
 |  
 |  Derive from this class to define new enumerations.
 |  
 |  Data descriptors defined here:
 |  
 |  name
 |      The name of the Enum member.
 |  
 |  value
 |      The value of the Enum member.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from EnumMeta:
 |  
 |  __members__
 |      Returns a mapping of member name->value.
 |      
 |      This mapping lists all enum members, including aliases. Note that this
 |      is a read-only view of the internal mapping.

